<a href="https://colab.research.google.com/github/srishtysuman/NLP/blob/master/Rating_prediction_on_music_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz

In [ ]:
import gensim
word_model = gensim.models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin.gz', binary=True)

#1.0 = 2506, #2.0=2712, #3.0=6108, #4.0=14926,#5.0=31983

Classes are highly imbalanced. So don't use accuracy as a measure. use confusion matrix or recall or precision or f1-score

In [ ]:
### function for loading files ###

def load_doc(filename):
    file = open(filename, 'r')
    text = file.readlines()
    file.close()
    return text

In [ ]:
#### loading files ####

texts_x=load_doc('digital_music_5.x.train.txt')
texts_y=load_doc('digital_music_5.y.train.txt')

In [ ]:
from sklearn.preprocessing import LabelEncoder

file1 = open('digital_music_5.y.train.txt', 'r') 
Lines = file1.readlines() 
count = 0
target1=[]
for line in Lines: 
    target1.append(float(line[0:-1]))
#le = LabelEncoder()
#le.fit(["1.0\n", "2.0\n","3.0\n","4.0\n","5.0\n"])
#list(le.classes_)
#labels=le.transform(target1)
target1[:290]

In [ ]:
#### tokenize the texts #####

from keras.preprocessing.text import Tokenizer
# tokenize
def tokenizer_(texts):
  texts = [w.lower() for w in texts]
  tokenizer = Tokenizer()
  tokenizer.fit_on_texts(texts)
  sequences = tokenizer.texts_to_sequences(texts)
  return sequences, tokenizer


In [ ]:
### tokenizing the train x ####

train_tokenized, tokenizer=tokenizer_(texts_x)

train_tokenized is the tokenized version of texts_x

In [ ]:
## getting the work index from tokenizer #####
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 129726 unique tokens.


In [ ]:
#### obtaining the maximum length of a sequence in tokenized sequence ########

maxl=0
for x in train_tokenized:
  t=len(x)
  if(t>maxl):
    maxl=t
print(maxl)
MAX_SEQUENCE_LENGTH=maxl
EMBEDDING_DIM=100

4518


In [ ]:
## pad trained tokenized sequences with 0 to obtain same length sequences ######

from keras.preprocessing.sequence import pad_sequences
data = pad_sequences(train_tokenized, maxlen=MAX_SEQUENCE_LENGTH)
print(data.shape)

(58235, 4518)


In [ ]:
from sklearn.model_selection import train_test_split
import tensorflow as tf

train_x=data
train_y=target1
X_train, X_val, Y_train, Y_val = train_test_split(train_x, train_y, test_size=0.20, random_state=42)
x_train = tf.convert_to_tensor(X_train, dtype=tf.float32)
y_train=tf.convert_to_tensor(Y_train, dtype=tf.float32)
x_val = tf.convert_to_tensor(X_val, dtype=tf.float32)
y_val=tf.convert_to_tensor(Y_val, dtype=tf.float32)

In [ ]:
from gensim.models import Word2Vec 
from gensim.models import KeyedVectors

In [ ]:
######### embedding index is a dictionary which contains vectors for each word in pretrained #####
import numpy as np

embeddings_index = {}
f = open('glove.6B.100d.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

In [ ]:
########### embedding matrix containing vectors corresponding to each word ###########

embedding_matrix = np.zeros((len(word_index) + 1, EMBEDDING_DIM))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        if(len(embedding_vector)<100):
          embedding_vector += [0.0] * (100 - len(embedding_vector))
        embedding_matrix[i] = embedding_vector

In [ ]:
embedding_matrix[122000]

In [ ]:
######### Embedding layer ###########

from keras.layers import Embedding

embedding_layer = Embedding(len(word_index) + 1,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=True)

In [ ]:
from keras import Input
from keras.layers import Conv1D, MaxPooling1D, Dense, Flatten
from keras import Model
import keras
from keras import Sequential

In [ ]:
len(y_train)

46588

In [ ]:
#### Model Definition #####
model = Sequential()
model.add(embedding_layer)
model.add(Conv1D(128, 5, padding='same', activation='relu'))
model.add(MaxPooling1D())
model.add(Flatten())
model.add(Dense(4158, activation='relu'))
model.add(Dense(250, activation='relu'))
model.add(Dense(6, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics='accuracy')
##keras.metrics.Recall()
model.summary()


In [ ]:
# Fit the model
from keras.utils import to_categorical

history = model.fit([train_x], y=to_categorical(train_y), batch_size=128, verbose=1, validation_split=0.25, shuffle=True, epochs=10)

In [ ]:
# Final evaluation of the model
scores = model.evaluate(x_val, y_val, verbose=0)
scores=[4]*len(x_val)
print("Recall: %.2f%%" % (scores[1]*100))

In [ ]:
yp=np.argmax(model.predict(x_val), axis=-1)
print(yp[:1000])

In [ ]:
from sklearn import metrics
matrix = metrics.confusion_matrix(y_val, yp)


In [ ]:
print(matrix)

In [ ]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

In [ ]:
estimator = KerasClassifier(build_fn = model, epochs = 10, batch_size = 10, verbose = 1)

In [ ]:
kfold = KFold(n_splits = 5, shuffle = True, random_state = 10)

In [ ]:
# Object to describe the result
results = cross_val_score(estimator, train_x, train_y, cv = kfold)
# Result
print("Result: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

In [ ]:
print(yp[:800])

In [ ]:
inputs = Input(shape=(MAX_LENGTH, ))

x = LSTM(64)(embedding_layer)
x = Dense(32, activation='relu')(x)
predictions = Dense(num_class, activation='softmax')(x)
model = Model(inputs=[inputs], outputs=predictions)
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['acc'])

model.summary()